In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
import sys
from tqdm import tqdm
import time
import keras.backend as K

2022-04-03 14:12:01.909417: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
sys.path.append('..')

In [3]:
import nets
from Data import datagen
import importlib 
import resnet1D_Ahmed
importlib.reload(datagen)  # Python 3.4+

<module 'Data.datagen' from '/home/keondopark/sleep/JupyterNotebooks/../Data/datagen.py'>

In [4]:
np.random.seed(1)

In [5]:
PROCESSED_DATA_PATH = os.path.join('/home','aiot','data','origin_npy')
save_signals_path_SC = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
save_annotations_path_SC = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')
save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST_filtered')
save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

In [6]:
def match_annotations_npy(dirname, filepath):
    filename = os.path.basename(filepath)
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]
    return filenames

In [7]:
dim_HT1D = (3000,1)
n_classes=6
epochs = 50
bs = 64
BASE_LEARNING_RATE = 1e-3
data_ratio = 0.25
PREV_CNT = 10
list_files_SC = [os.path.join(save_signals_path_SC, f) for f in os.listdir(save_signals_path_SC) if f.endswith('.npy')]
list_files_ST = [os.path.join(save_signals_path_ST, f) for f in os.listdir(save_signals_path_ST) if f.endswith('.npy')]

In [8]:
def read_csv_to_list(filepath):
    import csv
    with open(filepath, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        list_filepath = [row[0] for row in spamreader]
    return list_filepath

In [21]:
train_test_split = 0.7
split_cnt_SC = int(train_test_split * len(list_files_SC))
split_cnt_ST = int(train_test_split * len(list_files_ST))

list_files_train = []
list_files_test = []

list_ann_files_train = []
list_ann_files_test = []


list_files_SC_train = np.random.choice(list_files_SC[:split_cnt_SC], int(0.25 * split_cnt_SC), replace=False)
list_files_train += list_files_SC_train.tolist()
for f in list_files_SC_train:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_SC, ann_file[0]))

list_files_test += list_files_SC[split_cnt_SC:]

for f in list_files_SC[split_cnt_SC:]:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_SC, ann_file[0]))


list_files_ST_train = np.random.choice(list_files_ST[:split_cnt_ST], int(0.25 * split_cnt_ST), replace=False)
list_files_train += list_files_ST_train.tolist()
for f in list_files_ST_train:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_ST, ann_file[0]))

list_files_test += list_files_ST[split_cnt_ST:]
for f in list_files_ST[split_cnt_ST:]:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_ST, ann_file[0]))

train_generator = datagen.DataGenerator(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, balanced_sampling=True)
test_generator = datagen.DataGenerator(list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, balanced_sampling=False)

In [22]:
first_file = np.load(list_ann_files_train[0])

In [23]:
len(first_file)

2694

In [24]:
for i in train_generator.data_indexes[0]:
    print(first_file[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [20]:
train_generator.data_indexes

[array([1080, 2766,  551, ..., 2381, 1896, 1835]),
 array([  43,  324,  400, ..., 1244, 1286, 1255]),
 array([2153,   81, 2165, ..., 1154,  975,  971]),
 array([2329, 2377, 1911, ..., 1769, 1745, 1578]),
 array([ 556, 2395,  248, ..., 1697,  901,  876]),
 array([ 732,   51,   12, ..., 1662, 1827, 1264]),
 array([ 650,  128, 2613, ..., 1112, 1130, 1419]),
 array([2766, 2543, 2292, ..., 1060, 1066, 1320]),
 array([2436, 2596,  779, ..., 1099, 1775, 1811]),
 array([2455, 1481,  587, ..., 2065, 2138, 1766]),
 array([1949, 2146,  140, ..., 1531, 1046, 1203]),
 array([1784, 2588, 2464, ..., 1691, 1203, 1228]),
 array([ 179, 1971, 2109, ..., 1730, 1445, 1119]),
 array([1400, 2206, 2058, ..., 1903, 1897, 1636]),
 array([2029, 1884,  907, ..., 1065, 1037, 1370]),
 array([ 777, 2145,  646, ..., 1319, 1406, 1485]),
 array([ 838, 2455,  279, ..., 1847, 1425, 1386]),
 array([ 587, 1868,  240, ..., 1145, 1164, 1159]),
 array([ 631, 2453, 1287, ..., 1415, 1416, 1417]),
 array([ 568, 2075,  610, ..., 

In [50]:
bi

array([35,  7, 25, 42, 17, 44, 38, 20,  2, 32, 45,  1, 16,  4, 22, 49, 41,
       18, 43, 24,  0, 30, 31, 12,  3, 37, 13, 40, 33, 21, 47, 27, 15, 39,
       50, 10,  9, 48, 34, 23,  5, 36, 26, 19, 11,  8, 28, 46, 29, 14,  6])

In [20]:
# Calculate class weight
# Tested loss with class weight, but doesn't improve the accuracy

from collections import defaultdict
cnt_class = defaultdict(int)
for x, y in train_generator:
    unique, counts = np.unique(y, return_counts=True)
    for i, cnt in zip(unique, counts):
        cnt_class[i] += cnt
cnt_class_np = np.array(list(cnt_class.values()))
class_weight = sum(cnt_class_np)/(n_classes * cnt_class_np)


In [21]:
len(list_ann_files_train), len(list_ann_files_test)

(32, 58)

In [22]:
list_ann_files_train

['/home/aiot/data/origin_npy/annotations_SC/SC4732EJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4371FA-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4162EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4271FC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4131EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4412EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4722EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4141EU-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4581GM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4272FM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4772GC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4541FA-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4562FJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4401EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annot

In [28]:
model = modules.Conv1DAttention2()

In [29]:
x = np.random.random((1,3000,1))
x = tf.convert_to_tensor(x)
model(x)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.16749106, 0.1742762 , 0.15472369, 0.16053179, 0.17803392,
        0.1649434 ]], dtype=float32)>

In [30]:
model.summary()

Model: "conv1d_attention2_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_block_34 (conv1d_bloc multiple                  320       
_________________________________________________________________
conv1d_block_35 (conv1d_bloc multiple                  256       
_________________________________________________________________
conv1d_block_36 (conv1d_bloc multiple                  480       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 multiple                  0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 multiple                  0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 multiple                  0         
_________________________________________________________________
conv1d_block_37 (conv1d_bloc multiple          

In [31]:
def get_current_lr(epoch):
    lr = BASE_LEARNING_RATE
    for _ in range(epoch // 10):
        lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    optimizer.learning_rate = lr

In [32]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        bs = y_pred.shape[0]
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        #loss = -K.sum(loss, -1)
        loss = -K.sum(loss) / bs
        return loss
    
    return loss

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss_fn = weighted_categorical_crossentropy(weights=class_weight)

In [34]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './ckpt_Advanced_Conv1D', max_to_keep=1)

In [35]:
start_epoch = 0
#if manager.latest_checkpoint:
#    ckpt.restore(manager.latest_checkpoint)
#    start_epoch = ckpt.step.numpy()-1

In [36]:
best_test_acc = 0.0
for e in range(start_epoch, epochs):
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    print('-'*20, 'Epoch ' + str(e) + '-'*20)
    adjust_learning_rate(optimizer, e)
    start = time.time()
    for idx, (x, y) in enumerate(train_generator):   
        y_onehot = tf.one_hot(y, depth=n_classes)
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss = loss_fn(y_onehot, y_pred)
            #loss = loss_fn(y, y_pred)
        
        total_cnt += y_pred.shape[0]
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_loss += loss * y_pred.shape[0]
        if (idx + 1) % 10 == 0 or idx+1 == len(train_generator):
            print("[%d / %d] Training loss: %.6f, Training acc: %.3f"%
                  (idx+1, len(train_generator), total_loss / total_cnt, correct / total_cnt),end='\r', flush=True)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    print("")
    print("Training time: %.2f sec "%(time.time() - start))
    
    start = time.time()
    
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    for idx, (x, y) in enumerate(test_generator):
        y_pred = model(x, training=False)
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_cnt += y_pred.shape[0]
        y = tf.cast(y, dtype=tf.int32)
        y_onehot = tf.one_hot(y, depth=n_classes)
        #total_loss += loss_fn(y, y_pred).numpy() * y_pred.shape[0]
        total_loss += loss_fn(y_onehot, y_pred).numpy() * y_pred.shape[0]
            
        test_acc = correct / total_cnt
        test_loss = total_loss / total_cnt
        if (idx + 1) % 10 == 0 or idx+1 == len(test_generator):
            print("[%d / %d] test loss: %.6f, test accuracy: %.3f"%
                  (idx+1, len(test_generator), test_loss, test_acc),end='\r', flush=True)
    print("")
    print("Eval time: %.2f sec"%(time.time() - start))
    ckpt.step.assign_add(1)
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    

-------------------- Epoch 0--------------------
[1162 / 1162] Training loss: 3.180745, Training acc: 0.798
Training time: 130.09 sec 
[515 / 515] test loss: 146.889364, test accuracy: 0.709
Eval time: 27.83 sec
Saved checkpoint for step 2: ./ckpt_Advanced_Conv1D/ckpt-1
-------------------- Epoch 1--------------------
[1162 / 1162] Training loss: 3.385102, Training acc: 0.726
Training time: 130.57 sec 
[515 / 515] test loss: 56.402125, test accuracy: 0.661
Eval time: 27.20 sec
-------------------- Epoch 2--------------------
[1162 / 1162] Training loss: 3.588799, Training acc: 0.720
Training time: 129.54 sec 
[515 / 515] test loss: 158.990808, test accuracy: 0.682
Eval time: 29.97 sec
-------------------- Epoch 3--------------------
[1162 / 1162] Training loss: 3.572134, Training acc: 0.792
Training time: 129.64 sec 
[515 / 515] test loss: 153.431744, test accuracy: 0.764
Eval time: 29.66 sec
Saved checkpoint for step 5: ./ckpt_Advanced_Conv1D/ckpt-2
-------------------- Epoch 4-------

In [ ]:
correct, total_cnt, total_loss = 0.0, 0.0, 0.0
confusion_matrix = np.zeros((n_classes,n_classes))
for idx, (x, y) in enumerate(test_generator):
    y_pred = model(x, training=False)
    y_pred_cls = tf.math.argmax(y_pred, axis=-1)
    correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
    total_cnt += y_pred.shape[0]
    y = tf.cast(y, dtype=tf.int32)    
    for i in range(n_classes):
        for j in range(n_classes):
            confusion_matrix[i,j] += np.sum((y_pred_cls.numpy()==i) * (y.numpy()==j))


In [ ]:
for i in range(n_classes):
    print_ln = ""
    for j in range(n_classes):
        print_ln += "%.3f "%(confusion_matrix[i,j] / np.sum(confusion_matrix[i]))
        #print_ln += "%d "%(confusion_matrix[i,j])
    print(print_ln)

In [ ]:
x = np.arange(10)
np.random.seed(1)
np.random.shuffle(x)

In [ ]:
x

In [ ]:
y = np.arange(20)
np.random.shuffle(y)

In [ ]:
y

In [ ]:
pc = np.array([[0,0,1],[1,0,0],[0,1,0], [3,4,5]])

In [ ]:
n = pc.shape[0]
x = np.tile(np.expand_dims(pc,1), [1,n,1])
y = np.empty((n,n,3))
y[:] = np.tile(np.expand_dims(pc,0), [n,1,1])
dist = np.sum((x - y) ** 2, axis=2) ** 0.5 # n by n matrix

In [ ]:
instances_list = []
thr = 2
for i in range(1, n):    
    included = False
    for inst_set in instances_list:
        if i in inst_set:
            included = True
            break
    if not included:
        close_pts = set()           
        q = [i]
        while q:
            j = q.pop()
            #if j in close_pts: continue
            new_pts = set(np.where(dist[j,:] < thr)[0])
            add_pts = new_pts - close_pts
            q += list(add_pts)
            close_pts = close_pts.union(add_pts)
        
        instances_list.append(close_pts)
centroids = []
for s in instances_list:
    cent = np.mean(pc[list(s),:], axis=0)
    centroids.append(cent)
    
    

In [ ]:
centroids, instances_list

In [ ]:
x = np.array([[[1,2,3,4],[5,6,7,8],[4,3,2,1]], [[1,2,3,4],[5,9,7,8],[4,3,2,1]]])

In [ ]:
x.shape

In [ ]:
x[:,:,0] == 5

In [ ]:
np.sum((x[:,:,0] == 5) * (x[:,:,1] == 6))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a = np.random.random((2,3000))
b = np.ones(2)
c = 'abc'

In [ ]:
df = pd.DataFrame(a)
df['b'] = b
df['c'] = c

In [ ]:
df

In [ ]:
d = np.random.random((2,3000))
e = np.ones(2)*2
f = 'def'
df2 = pd.DataFrame(d)
df2['b'] = e
df2['c'] = f

In [ ]:
df2

In [ ]:
df.append(df2)